In [26]:
import os
import pandas as pd 
from bs4 import BeautifulSoup
from io import StringIO


In [12]:
SCORE_DIR = 'data/scores'

In [13]:
box_scores = os.listdir(SCORE_DIR)

In [14]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith('.html')]

In [16]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select('tr.over_header')]
    [s.decompose() for s in soup.select('tr.thead')]
    return soup

In [24]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs={'id' : 'line_score'})[0]

In [31]:
box_score = box_scores[0]
soup = parse_html(box_score)
pd.read_html(StringIO(str(soup)), attrs={"id" : "line_score"})[0]

,Unnamed: 0,1,2,3,4,T
0,ATL,22,27,27,22,98
1,DAL,23,21,28,23,95
